# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.234610987851                   -0.50    7.0         
  2   -7.250077974306       -1.81       -1.39    1.0   6.84ms
  3   -7.251037013149       -3.02       -1.95    2.0   7.79ms
  4   -7.251107618298       -4.15       -1.95    2.0   8.75ms
  5   -7.251321251412       -3.67       -2.51    1.0   7.18ms
  6   -7.251337426835       -4.79       -3.02    1.0   7.25ms
  7   -7.251338622488       -5.92       -3.67    1.0   7.46ms
  8   -7.251338793341       -6.77       -4.07    3.0   10.4ms
  9   -7.251338798030       -8.33       -4.58    1.0   7.19ms
 10   -7.251338798568       -9.27       -5.18    1.0   7.60ms
 11   -7.251338798696       -9.90       -5.41    3.0   9.87ms
 12   -7.251338798703      -11.16       -5.76    1.0   7.57ms
 13   -7.251338798704      -12.00       -5.98    2.0   9.37ms
 14   -7.251338798704      -12.07       -6.50    1.0   7.54ms
 15   -7.

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.07867752414303222
[ Info: Arnoldi iteration step 2: normres = 0.19960763632382306
[ Info: Arnoldi iteration step 3: normres = 0.5015881419044331
[ Info: Arnoldi iteration step 4: normres = 0.722677225440526
[ Info: Arnoldi iteration step 5: normres = 0.8081766824928922
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (4.88e-01, 9.52e-02, 6.05e-01, 1.98e-01, 2.43e-02)
[ Info: Arnoldi iteration step 6: normres = 0.1905211745451693
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (7.17e-02, 5.31e-02, 1.04e-01, 1.10e-01, 3.13e-02)
[ Info: Arnoldi iteration step 7: normres = 0.07166764053005673
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (2.45e-03, 3.36e-03, 6.87e-03, 4.81e-02, 4.87e-02)
[ Info: Arnoldi iteration step 8: normres = 0.09674657993730995
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (1.01e-0